In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
cudf 24.8.3 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires py

In [2]:
!huggingface-cli login --token yout_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [4]:
torch.cuda.get_device_capability()[0]

6

In [5]:
df = pd.read_csv('/kaggle/input/fincausal2025/training_data_en.csv',sep=';',on_bad_lines='skip')

In [6]:
df

,ID,Text,Question,Answer
0,3335,"Nationwide is in robust financial health, havi...",What is the effect of achieving profits of ove...,Nationwide is in robust financial health
1,3336,All the Directors are resident in the UK and t...,What is the impact of the Company's small size...,the Board considers it sensible for all the Di...
2,3337,"Overall, Group trading continues to be subdued...",What is the main reason why the Group trading ...,legacy issues
3,3341,Included within the Group's administrative exp...,What is the effect of the Group taking a contr...,consolidate the results of a portfolio company...
4,3343,The following points were noted: • A review of...,What happened because of management performing...,The outstanding receivables are fully impaired
...,...,...,...,...
1991,5094,"Because the ""curing"" of concrete takes place o...","What are the consequences of the ""curing"" of c...",some of these tests span periods up to 12 mont...
1992,3568.a,Finally it is worth noting that in addition to...,What was the impact of the continuing enhanced...,has made us the biggest in our industry
1993,5044.1,The Group will operate in highly competitive m...,What explains the rapid evolution and adaptati...,"factors such as consumer tastes, trends and te..."
1994,1463,Any increases to Non-Executive Director fees w...,What factors will influence the decision to in...,the outcome of a review process and taking int...


In [7]:
train_prompt = lambda cntxt,inpt,response: f"""<bos><start_of_turn>user
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction: You are an AI assistant specialized in Finance Causal extraction. Your task is to identify and return either the cause or effect as requested, verbatim, from the provided financial text in ### Context.

### Context: {{cntxt}}
### Input: {{inpt}}
<end_of_turn>
<start_of_turn>model
{{response}} <end_of_turn>""".format(cntxt=cntxt,inpt=inpt,response=response)

In [8]:
test_prompt = lambda cntxt,inpt: f"""<bos><start_of_turn>user
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction: You are an AI assistant specialized in Finance Causal extraction. Your task is to identify and return either the cause or effect as requested, verbatim, from the provided financial text in ### Context.

### Context: {{cntxt}}
### Input: {{inpt}}
<end_of_turn>
<start_of_turn>model  """.format(cntxt=cntxt,inpt=inpt)

In [9]:
# final_prompt = prompt.format(cntxt='hi',inpt='okk',response="your response here")

In [10]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
df_train, df_test = train_test_split(df, test_size=0.20, random_state=42)

In [13]:
df_train['prompt'] = df_train.progress_apply(lambda x: train_prompt(x['Text'],x['Question'],x['Answer']),axis=1)
df_test['prompt'] = df_test.progress_apply(lambda x: test_prompt(x['Text'],x['Question']),axis=1)

  0%|          | 0/1596 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

In [14]:
# https://huggingface.co/google/gemma-7b/discussions/62

# <bos><start_of_turn>user
# Instruction + Context + Input <end_of_turn>
# <start_of_turn>model
# Reponse<end_of_turn>

In [15]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

In [16]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)


In [17]:
# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2-9b-it"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [18]:
# Load your dataset (replace 'your_dataset_name' and 'split_name' with your actual dataset information)
# dataset = load_dataset("your_dataset_name", split="split_name")
# dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")

In [19]:
dataset = Dataset.from_pandas(df_train)
# dataset = Dataset.from_pandas(df)

In [20]:
dataset

Dataset({
    features: ['ID', 'Text', 'Question', 'Answer', 'prompt', '__index_level_0__'],
    num_rows: 1596
})

In [21]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/1596 [00:00<?, ? examples/s]

In [22]:
dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset["train"]
test_data = dataset["test"]

In [23]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
) 
model = get_peft_model(model, lora_config)

In [24]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


In [25]:
#new code using SFTTrainer
import transformers
from transformers import TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()


# Tokenizer adjustments if needed
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

# Define SFTTrainer with early stopping and best model loading
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        num_train_epochs=5,                    # Define epochs as needed
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        evaluation_strategy="epoch",           # Evaluate at the end of each epoch
        load_best_model_at_end=True,           # Load the best model after training
        metric_for_best_model="eval_loss",     # Use evaluation loss to determine the best model
        report_to='none'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stops if eval metric doesn’t improve for 2 epochs
)


# trainer = SFTTrainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     dataset_text_field="prompt",
#     peft_config=lora_config,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         warmup_steps=0.03,
# #         max_steps=100,
#         num_train_epochs=5,
#         learning_rate=2e-4,
#         logging_steps=5,
#         output_dir="outputs",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#         report_to='none'
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1436 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [26]:
# Start the training process
trainer.train()

new_model = "gemma-2b-it-QA-Tuned" #Name of the model you will be pushing to huggingface model hub
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

Epoch,Training Loss,Validation Loss
1,1.051300,1.103516
2,0.684600,1.120117
3,0.384800,1.259766
4,0.141500,1.478516


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in

In [27]:
# Merge the model with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/avinasht/gemma-2b-it-QA-Tuned/commit/fd54c906f2fbb847c5266c14ecc21142a4a35590', commit_message='Upload tokenizer', commit_description='', oid='fd54c906f2fbb847c5266c14ecc21142a4a35590', pr_url=None, repo_url=RepoUrl('https://huggingface.co/avinasht/gemma-2b-it-QA-Tuned', endpoint='https://huggingface.co', repo_type='model', repo_id='avinasht/gemma-2b-it-QA-Tuned'), pr_revision=None, pr_num=None)

In [29]:
def get_completion(prompt, model, tokenizer):
    device = "cuda:0"
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=10, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    # decoded = tokenizer.batch_decode(generated_ids)
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return (decoded)

In [30]:
df_test.reset_index(drop=True,inplace=True)

In [31]:
get_completion(df_test['prompt'][0],merged_model,tokenizer)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


"user\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: You are an AI assistant specialized in Finance Causal extraction. Your task is to identify and return either the cause or effect as requested, verbatim, from the provided financial text in ### Context.\n\n### Context: The Audit and Risk Committee has carried out its annual assessment of the internal controls of the Fund's service providers for the year ended 30th June 2017 and considered the internal control procedures to be adequate based on the findings of their respective ISAE 3402 or SSAE 16 reports.\n### Input: What factor led the Audit and Risk Committee to consider the internal control procedures to be adequate?\n\nmodel   The findings of their respective ISAE 34"

In [32]:
get_completion(df_test['prompt'][1],merged_model,tokenizer)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'user\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: You are an AI assistant specialized in Finance Causal extraction. Your task is to identify and return either the cause or effect as requested, verbatim, from the provided financial text in ### Context.\n\n### Context: We are focused on the need to maximise the effectiveness of our information systems and technology as a business enabler and to reduce both cost and exposure as a result.\n### Input: Why are they going to reduce both cost and exposure?\n\nmodel   We are focused on minimizing the impact of technology disruption'

In [33]:
get_completion(df_test['prompt'][2],merged_model,tokenizer)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'user\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: You are an AI assistant specialized in Finance Causal extraction. Your task is to identify and return either the cause or effect as requested, verbatim, from the provided financial text in ### Context.\n\n### Context: Earnings per share Earnings per share were 117.2 pence (2016: 78.7 pence), the near 50% increase being higher than the increase in pre-tax profit as a result of a lower effective rate of taxation in 2017.\n### Input: What impact did the lower effective rate of taxation in 2017 have?\n\nmodel   Earnings per share were 117.2'

In [34]:
get_completion(df_test['prompt'][3],merged_model,tokenizer)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'user\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: You are an AI assistant specialized in Finance Causal extraction. Your task is to identify and return either the cause or effect as requested, verbatim, from the provided financial text in ### Context.\n\n### Context: SSE worked throughout 2016/17 to actively promote the Speak Up policy, alongside awareness raising of Doing the right thing: A guide to ethical business conduct for SSE employees. As a result, the number of whistle-blowing reports has more than doubled between 2015/16 and 2016/17 from 41 to 88.\n### Input: What motivated the number of whistle-blowers to more than double from 41 in 2015/16 to 88 in 2016/17?\n\nmodel   SSE worked throughout 2016/1'

In [35]:
# apply on entire dataset
# df_test['predicted'] = df_test['prompt'].progress_apply(lambda x:get_completion(x,merged_model,tokenizer))

In [36]:
# https://adithyask.medium.com/a-beginners-guide-to-fine-tuning-gemma-0444d46d821c